In [19]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import io
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [9]:
DIR = '/datasets/dsc180a-wi20-public/Malware/group_data/group_02/sensitive_data/processed/infomax'
fp = '/datasets/dsc180a-wi20-public/Malware/group_data/group_02/sensitive_data/interim/graph/graph.mat'

In [10]:
data = io.loadmat(fp)
embedding = torch.load(osp.join(DIR, 'embedding.pt'))

In [11]:
X = embedding.cpu().numpy()
y = data['post_label'].reshape(-1,)
post_mask = data['post_indx'].reshape(-1,)
X= X[post_mask, :]

In [12]:
def evaluate(clf, X_train, y_train, X_test, y_test):
    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
    ]   
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    res = {}
    for i in METRICS:
        res[i.name] = i(y_test, y_pred).numpy()
    return res

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
neg, pos = np.bincount(y)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
clfs = [
        LogisticRegression(
        verbose=False, max_iter=2000, class_weight = class_weight, n_jobs=8
        ),
        LinearSVC(class_weight=class_weight),
        RandomForestClassifier(class_weight=class_weight, n_jobs=8)
    ]

Examples:
    Total: 103413
    Positive: 4409 (4.26% of total)

Weight for class 0: 0.52
Weight for class 1: 11.73


In [17]:
res = {}
for clf in clfs:
    res[clf.__class__.__name__] = evaluate(clf, X_train, y_train, X_test, y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [21]:
pd.DataFrame(res).T

,tp,fp,tn,fn,accuracy,precision,recall,auc
LogisticRegression,538.0,5876.0,13888.0,381.0,0.697481,0.083879,0.585419,0.644055
LinearSVC,560.0,5680.0,14084.0,359.0,0.708021,0.089744,0.609358,0.660983
RandomForestClassifier,241.0,2081.0,17683.0,678.0,0.866605,0.103790,0.262242,0.578475
